# Ingest Batch

Ingest a batch of experiments into structured numpy arrays and json meta data files into PRP S3

See [file formats](https://github.com/braingeneers/braingeneers/wiki/File-Formats) for details on the source file format and [layout](https://github.com/braingeneers/braingeneers/wiki/data-repository-layout) for details on the repository structure.

Any .txt files will be added to a "notes" field and any .json files will be combined into the releavant metadata.

Note: The original Intan code saves the data as float64 which effectively increaes the size of the data by 4x vs its original source as 16 bit samples. The below code is modified to keep things in 16 bit form and as a result does not scale the amplifier_data structure leaving that to the analysis phase. There is still significant overhead added vs. the raw file (82M vs. 44M) but its much better then just storing the float64

In [1]:
import os
import re
import sys
import time
import math
import struct
import glob
import json
import datetime
import pprint
import functools
import requests
import numpy as np


# NOTE: Must import before we change to the archive directory...
from intanutil.read_header import read_header
from intanutil.get_bytes_per_data_block import get_bytes_per_data_block
from intanutil.read_one_data_block import read_one_data_block
from intanutil.notch_filter import notch_filter
from intanutil.data_to_result import data_to_result

In [2]:
def read_data(filename):
    """
    Reads an Intan Technologies RHD2000 data file generated by the evaluation board GUI.
    Returns data in a dictionary in raw sample units
    
    Adapted from Intan sample code...
    Michael Gibson 17 July 2015
    Modified Adrian Foy Sep 2018   
    """

    tic = time.time()
    fid = open(filename, 'rb')
    filesize = os.path.getsize(filename)

    header = read_header(fid)

    print('{} amplifier channels'.format(header['num_amplifier_channels']))
    print('{} auxiliary input channels'.format(header['num_aux_input_channels']))
    print('{} supply voltage channels'.format(header['num_supply_voltage_channels']))
    print('{} board ADC channels'.format(header['num_board_adc_channels']))
    print('{} board digital input channels'.format(header['num_board_dig_in_channels']))
    print('{} board digital output channels'.format(header['num_board_dig_out_channels']))
    print('{} temperature sensors channels'.format(header['num_temp_sensor_channels']))

    # Determine how many samples the data file contains.
    bytes_per_block = get_bytes_per_data_block(header)

    # How many data blocks remain in this file?
    data_present = False
    bytes_remaining = filesize - fid.tell()
    if bytes_remaining > 0:
        data_present = True

    if bytes_remaining % bytes_per_block != 0:
        raise Exception('Something is wrong with file size : should have a whole number of data blocks')

    num_data_blocks = int(bytes_remaining / bytes_per_block)

    num_amplifier_samples = header['num_samples_per_data_block'] * num_data_blocks
    num_aux_input_samples = int((header['num_samples_per_data_block'] / 4) * num_data_blocks)
    num_supply_voltage_samples = 1 * num_data_blocks
    num_board_adc_samples = header['num_samples_per_data_block'] * num_data_blocks
    num_board_dig_in_samples = header['num_samples_per_data_block'] * num_data_blocks
    num_board_dig_out_samples = header['num_samples_per_data_block'] * num_data_blocks

    record_time = num_amplifier_samples / header['sample_rate']

    if data_present:
        print('File contains {:0.3f} seconds of data.  Amplifiers were sampled at {:0.2f} kS/s.'.format(record_time, header['sample_rate'] / 1000))
    else:
        print('Header file contains no data.  Amplifiers were sampled at {:0.2f} kS/s.'.format(header['sample_rate'] / 1000))

    if data_present:
        # Pre-allocate memory for data.
        print('')
        print('Allocating memory for data...')

        data = {}
        if (header['version']['major'] == 1 and header['version']['minor'] >= 2) or (header['version']['major'] > 1):
            data['t_amplifier'] = np.zeros(num_amplifier_samples, dtype=np.int)
        else:
            data['t_amplifier'] = np.zeros(num_amplifier_samples, dtype=np.uint)

        # NOTE: Changed from uint to uint16
        data['amplifier_data'] = np.zeros([header['num_amplifier_channels'], num_amplifier_samples], dtype=np.uint16)
        data['aux_input_data'] = np.zeros([header['num_aux_input_channels'], num_aux_input_samples], dtype=np.uint)
        data['supply_voltage_data'] = np.zeros([header['num_supply_voltage_channels'], num_supply_voltage_samples], dtype=np.uint)
        data['temp_sensor_data'] = np.zeros([header['num_temp_sensor_channels'], num_supply_voltage_samples], dtype=np.uint)
        data['board_adc_data'] = np.zeros([header['num_board_adc_channels'], num_board_adc_samples], dtype=np.uint)
        
        # by default, this script interprets digital events (digital inputs and outputs) as booleans
        # if unsigned int values are preferred(0 for False, 1 for True), replace the 'dtype=np.bool' argument with 'dtype=np.uint' as shown
        # the commented line below illustrates this for digital input data; the same can be done for digital out
        
        #data['board_dig_in_data'] = np.zeros([header['num_board_dig_in_channels'], num_board_dig_in_samples], dtype=np.uint)
        data['board_dig_in_data'] = np.zeros(
            [header['num_board_dig_in_channels'], num_board_dig_in_samples], dtype=np.bool)
        data['board_dig_in_raw'] = np.zeros(num_board_dig_in_samples, dtype=np.uint)
        
        data['board_dig_out_data'] = np.zeros(
            [header['num_board_dig_out_channels'], num_board_dig_out_samples], dtype=np.bool)
        data['board_dig_out_raw'] = np.zeros(num_board_dig_out_samples, dtype=np.uint)

        # Read sampled data from file.
        print('Reading data from file...')

        # Initialize indices used in looping
        indices = {}
        indices['amplifier'] = 0
        indices['aux_input'] = 0
        indices['supply_voltage'] = 0
        indices['board_adc'] = 0
        indices['board_dig_in'] = 0
        indices['board_dig_out'] = 0

        print_increment = 10
        percent_done = print_increment
        for i in range(num_data_blocks):
            read_one_data_block(data, header, indices, fid)

            # Increment indices
            indices['amplifier'] += header['num_samples_per_data_block']
            indices['aux_input'] += int(header['num_samples_per_data_block'] / 4)
            indices['supply_voltage'] += 1
            indices['board_adc'] += header['num_samples_per_data_block']
            indices['board_dig_in'] += header['num_samples_per_data_block']
            indices['board_dig_out'] += header['num_samples_per_data_block']            

            fraction_done = 100 * (1.0 * i / num_data_blocks)
            if fraction_done >= percent_done:
                print('.', end='', flush=True)
#                 print('{}% done...'.format(percent_done))
                percent_done = percent_done + print_increment

        # Make sure we have read exactly the right amount of data.
        bytes_remaining = filesize - fid.tell()
        if bytes_remaining != 0: raise Exception('Error: End of file not reached.')

    # Close data file.
    fid.close()

    if (data_present):
        print('Parsing data...')

        # Extract digital input channels to separate variables.
        for i in range(header['num_board_dig_in_channels']):
            data['board_dig_in_data'][i, :] = np.not_equal(np.bitwise_and(data['board_dig_in_raw'], (1 << header['board_dig_in_channels'][i]['native_order'])), 0)

        # Extract digital output channels to separate variables.
        for i in range(header['num_board_dig_out_channels']):
            data['board_dig_out_data'][i, :] = np.not_equal(np.bitwise_and(data['board_dig_out_raw'], (1 << header['board_dig_out_channels'][i]['native_order'])), 0)

        # Scale voltage levels appropriately.
        # NOTE: Commented out to reduce size of file by 4x by storing 16 bit ints in the resulting .npy
#         data['amplifier_data'] = np.multiply(0.195, (data['amplifier_data'].astype(np.int32) - 32768))      # units = microvolts
        data['aux_input_data'] = np.multiply(37.4e-6, data['aux_input_data'])               # units = volts
        data['supply_voltage_data'] = np.multiply(74.8e-6, data['supply_voltage_data'])     # units = volts
        if header['eval_board_mode'] == 1:
            data['board_adc_data'] = np.multiply(152.59e-6, (data['board_adc_data'].astype(np.int32) - 32768)) # units = volts
        elif header['eval_board_mode'] == 13:
            data['board_adc_data'] = np.multiply(312.5e-6, (data['board_adc_data'].astype(np.int32) - 32768)) # units = volts
        else:
            data['board_adc_data'] = np.multiply(50.354e-6, data['board_adc_data'])           # units = volts
        data['temp_sensor_data'] = np.multiply(0.01, data['temp_sensor_data'])               # units = deg C

        # Check for gaps in timestamps.
        num_gaps = np.sum(np.not_equal(data['t_amplifier'][1:]-data['t_amplifier'][:-1], 1))
        assert num_gaps == 0  # We don't handle missing samples in all our downstream analysis

        # Scale time steps (units = seconds).
        data['t_amplifier'] = data['t_amplifier'] / header['sample_rate']
        data['t_aux_input'] = data['t_amplifier'][range(0, len(data['t_amplifier']), 4)]
        data['t_supply_voltage'] = data['t_amplifier'][range(0, len(data['t_amplifier']), header['num_samples_per_data_block'])]
        data['t_board_adc'] = data['t_amplifier']
        data['t_dig'] = data['t_amplifier']
        data['t_temp_sensor'] = data['t_supply_voltage']

        # If the software notch filter was selected during the recording, apply the
        # same notch filter to amplifier data here.
        assert header['notch_filter_frequency'] == 0
        if header['notch_filter_frequency'] > 0:
            print('Applying notch filter...')

            print_increment = 10
            percent_done = print_increment
            for i in range(header['num_amplifier_channels']):
                data['amplifier_data'][i,:] = notch_filter(data['amplifier_data'][i,:], header['sample_rate'], header['notch_filter_frequency'], 10)

                fraction_done = 100 * (i / header['num_amplifier_channels'])
                if fraction_done >= percent_done:
                    print('{}% done...'.format(percent_done))
                    percent_done += print_increment
    else:
        data = [];

    # Move variables to result struct.
    result = data_to_result(header, data, data_present)

    print('Done!  Elapsed time: {0:0.1f} seconds'.format(time.time() - tic))
    return result

@functools.lru_cache(maxsize=128)
def read_data_cached(path):
    return read_data(path)

print(read_data_cached.cache_info())
print("read_data cache cleared.")
read_data_cached.cache_clear()

CacheInfo(hits=0, misses=0, maxsize=128, currsize=0)
read_data cache cleared.


In [3]:
read_data_cached.cache_info()

CacheInfo(hits=0, misses=0, maxsize=128, currsize=0)

## Ingest
Run from here down after setting the UUID and Issue to actually ingest and sync up to PRP S3

In [14]:
os.chdir("/public/groups/braingeneers/archive")

# Batch = set of folders each with an experiment and all its recordings
batch_uuid = "2019-02-05-v2"
batch_issue = "https://github.com/braingeneers/internal/issues/13"

In [16]:
print("Ingesting batch {}".format(batch_uuid))

os.makedirs("derived/{}".format(batch_uuid), exist_ok=True)

batch_metadata = {
    "uuid": batch_uuid,
    "issue": batch_issue,
    "notes": "\n".join([open(f).read() for f in glob.glob("original/{}/*.txt".format(batch_uuid))]),
    "experiments": []
}

# Add any metadata from json files to the batch metadata
for f in glob.glob("original/{}/*.json".format(batch_uuid)):
    batch_metadata.update(json.load(open(f)))
    
# Ingest each experiment and exhaust in normalized form into derived/<batch_uuid>/
for original_path in sorted(glob.glob("original/{}/*/".format(batch_uuid)))[0:1]:

    experiment_metadata = {}
    
    experiment_metadata["name"] = os.path.basename(os.path.normpath(original_path)).replace(" ","-")
    print("Experiment {} from {}".format(experiment_metadata["name"], original_path))
    
    # Concatenate all *.txt files into a "notes" field
    experiment_metadata["notes"] = "\n".join(
        [open(f).read() for f in glob.glob("{}/*.txt".format(original_path))])
        
    # Walk through each *.rhd in the experiment sorted alphabetically and therefore by time
    experiment_metadata["samples"] = [] 
    for sample_path in sorted(glob.glob("{}/*.rhd".format(original_path)))[0:1]:

        print("Sample {}".format(sample_path))
        
        # Try reading its *.rhd file stopping if there is an error. Any earlier
        # samples in the run will be retained and stored but with an "error" field
        # in the experiments metadata
        try:
            data = read_data_cached(sample_path)
        except Exception as e:
            experiment_metadata["error"] = "{}: {}".format(sample_path, str(e))
            print("ERROR: {}".format(experiment_metadata["error"]))
            break

        sample_metadata = {}
        
        sample_metadata["name"] = os.path.splitext(os.path.basename(sample_path).replace(" ","-"))[0]

        sample_metadata["timestamp"] = datetime.datetime.strptime(
            re.findall(r"_(\d+_\d+)", sample_metadata["name"])[0], "%y%m%d_%H%M%S").isoformat()

        if "timestamp" not in batch_metadata:
            batch_metadata["timestamp"] = sample_metadata["timestamp"]

        # Add all the Intan metadata - < is a hack to exclude all sample data
        sample_metadata.update({k: v for k, v in data.items() if sys.getsizeof(v) < 2048})
        
        sample_metadata["original"] = sample_path
        sample_metadata["derived"] = "derived/{}/{}.npy".format(batch_uuid, sample_metadata["name"])

        # Save the original un-modified np.uint16 to save space and so there is no loss of accuracy
        sample_metadata["units"] = "µV"
        sample_metadata["offset"] = 32768
        sample_metadata["scaler"] = 0.195
        np.save(sample_metadata["derived"], data["amplifier_data"])

        experiment_metadata["samples"].append(sample_metadata)

    # Add any metadata from json files to the experiment metadata
    for f in glob.glob("{}/*.json".format(original_path)):
        experiment_metadata.update(json.load(open(f)))

    # Save the meta data for this experiment
    print("NAME", experiment_metadata["name"])
    with open("derived/{}/{}.json".format(batch_uuid, experiment_metadata["name"]), "w") as f:
        json.dump(experiment_metadata, f, sort_keys=True)

    batch_metadata["experiments"].append("derived/{}/{}.json".format(
        batch_uuid, experiment_metadata["name"]))
    
with open("derived/{}/metadata.json".format(batch_uuid), "w") as f:
    json.dump(batch_metadata, f, sort_keys=True)

print("Finished ingesting batch.")

Ingesting batch 2019-02-05-v2
Experiment OrganoidTest1 from original/2019-02-05-v2/OrganoidTest1/
Sample original/2019-02-05-v2/OrganoidTest1/OganoidTest1_190205_193542.rhd
NAME OrganoidTest1
Finished ingesting batch.


In [ ]:
# # Mirror ALL BATCHES AND EXPERIMENTS of local to PRP WITH DELETE - DANGER
# !aws --profile {os.getenv("AWS_PROFILE")} --endpoint {os.getenv("AWS_S3_ENDPOINT")} \
#     s3 sync /public/groups/braingeneers/archive/ s3://braingeneers/archive/ \
#     --delete --acl public-read --dryrun

In [ ]:
# # Remove all derived in PRP S3 from this current ingest
# !aws --profile {os.getenv("AWS_PROFILE")} --endpoint {os.getenv("AWS_S3_ENDPOINT")} \
#     s3 rm --recursive s3://braingeneers/archive/derived/{batch_uuid}/

In [11]:
# Mirror the derived data for this batch to PRP S3 WITH delete
!aws --profile {os.getenv("AWS_PROFILE")} --endpoint {os.getenv("AWS_S3_ENDPOINT")} \
    s3 sync /public/groups/braingeneers/archive/derived/'{batch_uuid}' \
    s3://braingeneers/archive/derived/{batch_uuid} \
    --delete --acl public-read

upload: derived/2019-02-05/OganoidTest1_190205_193642.npy to s3://braingeneers/archive/derived/2019-02-05/OganoidTest1_190205_193642.npy
upload: derived/2019-02-05/OrganoidTest1.json to s3://braingeneers/archive/derived/2019-02-05/OrganoidTest1.json
upload: derived/2019-02-05/OganoidTest1_190205_193542.npy to s3://braingeneers/archive/derived/2019-02-05/OganoidTest1_190205_193542.npy
upload: derived/2019-02-05/OrganoidTestStimulate1.json to s3://braingeneers/archive/derived/2019-02-05/OrganoidTestStimulate1.json
upload: derived/2019-02-05/OganoidTest1_190205_193842.npy to s3://braingeneers/archive/derived/2019-02-05/OganoidTest1_190205_193842.npy
upload: derived/2019-02-05/OganoidTest1_190205_193742.npy to s3://braingeneers/archive/derived/2019-02-05/OganoidTest1_190205_193742.npy
upload: derived/2019-02-05/OganoidTest1_190205_193942.npy to s3://braingeneers/archive/derived/2019-02-05/OganoidTest1_190205_193942.npy
upload: derived/2019-02-05/OrganoidTestStimulate1_190205_194659.npy to 

In [12]:
# List all the derived files uploaded to PRP S3 for this batch of experiments
!aws --profile {os.getenv("AWS_PROFILE")} --endpoint {os.getenv("AWS_S3_ENDPOINT")} \
    s3 ls --recursive s3://braingeneers/archive/derived/{batch_uuid}

2019-02-06 07:39:58   76815488 archive/derived/2019-02-05/OganoidTest1_190205_193542.npy
2019-02-06 07:39:58   76815488 archive/derived/2019-02-05/OganoidTest1_190205_193642.npy
2019-02-06 07:39:59   76815488 archive/derived/2019-02-05/OganoidTest1_190205_193742.npy
2019-02-06 07:39:59   76815488 archive/derived/2019-02-05/OganoidTest1_190205_193842.npy
2019-02-06 07:39:59   70041728 archive/derived/2019-02-05/OganoidTest1_190205_193942.npy
2019-02-06 07:39:58      72970 archive/derived/2019-02-05/OrganoidTest1.json
2019-02-06 07:39:58      87840 archive/derived/2019-02-05/OrganoidTestStimulate1.json
2019-02-06 07:40:02   76815488 archive/derived/2019-02-05/OrganoidTestStimulate1_190205_194659.npy
2019-02-06 07:40:03   76815488 archive/derived/2019-02-05/OrganoidTestStimulate1_190205_194759.npy
2019-02-06 07:40:03   76815488 archive/derived/2019-02-05/OrganoidTestStimulate1_190205_194859.npy
2019-02-06 07:40:04   76815488 archive/derived/2019-02-05/OrganoidTestStimulate1_190205_194959.